In [1]:
!pip install scikit-learn -U
!pip install stable_baselines3

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.7/dist-packages (0.24.2)


In [7]:
from scipy.io import loadmat
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from scipy.linalg import eigh, pinv

from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import mean_squared_error

import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO, DQN, A2C
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [8]:
DATASET_PATH = '/content/drive/MyDrive/Brain-Computer Interfaces/Motor Imagery Dataset/'
# mat = loadmat(DATASET_PATH + 'data_set_IVa_al.mat')
mat = loadmat(DATASET_PATH + 'BCICIV_calib_ds1e_100Hz.mat')

In [9]:
cnt = mat['cnt'] * 0.1
mrk = mat['mrk']
nfo = mat['nfo']
x = [each[0] for each in nfo['xpos'][0][0]]
y = [each[0] for each in nfo['ypos'][0][0]]
trials = mrk[0][0][0][0]
labels = mrk[0][0][1][0]
fs = nfo['fs'][0][0][0][0]
channels = [each[0] for each in nfo['clab'][0][0][0]]
channel_poses = [[x[i], y[i]] for i in range(len(x))]
# label_names = [each[0] for each in nfo['classes'][0][0][0]]
print('cnt.shape', cnt.shape)
print('trials', trials[:10], 'trials.shape', trials.shape)
print('labels', labels[:10], 'labels.shape', labels.shape)
print('fs', fs)
print('channels', channels)
# print('label_names', label_names)

cnt.shape (190330, 59)
trials [2092 2892 3692 4492 5292 6092 6892 7692 8492 9292] trials.shape (200,)
labels [ 1  1 -1  1 -1 -1 -1 -1 -1  1] labels.shape (200,)
fs 100
channels ['AF3', 'AF4', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'CFC7', 'CFC5', 'CFC3', 'CFC1', 'CFC2', 'CFC4', 'CFC6', 'CFC8', 'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8', 'CCP7', 'CCP5', 'CCP3', 'CCP1', 'CCP2', 'CCP4', 'CCP6', 'CCP8', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'PO1', 'PO2', 'O1', 'O2']


In [10]:
def plot_magnitude_spectrum(sig, sr, title='signal', color='b'):
    n = len(sig)
    rf = np.linspace(0, sr / 2, num=round(n/2))
    fx = np.fft.fft(sig)
    fx = fx[:round(len(sig)/2)]                 # select half of all coeficients 
    fx = np.absolute(fx)                        # caculate magnitudes
    plt.figure(figsize=(18, 5))
    plt.plot(rf, fx, color=color)
    plt.xlabel('Frequency (Hz)')
    plt.title(title)
    plt.show()

def butter_bandpass_filter(data, lowcut, highcut, fs, order):
    banded_signals = []
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='bandpass')
    for i in range(data.shape[1]):
        y = signal.filtfilt(b, a, data[:, i])
        banded_signals.append(y)
    return np.array(banded_signals).T

def calculate_euclidean_distance(channel_poses, ci):
    distances = []
    for i in range(len(channel_poses)):
        score = np.sqrt(sum((np.array(channel_poses[i]) - np.array(ci)) ** 2))
        distances.append([i, score])    
        
    return sorted(distances, key=lambda x: x[1])[1:]


def find_laplacian_neighbours(channel_poses, dis, mode):
    if mode == 'low':
        cuton, cutoff = 0, 8
    elif mode == 'high':
        cuton, cutoff = 10, 22
    low_laplacian_neighbours_candidates = []
    for i, each in enumerate(channel_poses):
        for idx, d in dis[cuton:cutoff]:
            if i == idx:
                low_laplacian_neighbours_candidates.append(each + [d, idx])
                break

    for i in range(len(low_laplacian_neighbours_candidates)):
        cand_x, cand_y, d, idx = low_laplacian_neighbours_candidates[i]
        x_dis = np.sqrt((float(cand_x) - ci[0]) ** 2)
        y_dis = np.sqrt((float(cand_y) - ci[1]) ** 2)
        low_laplacian_neighbours_candidates[i] += [x_dis, y_dis]
    
    x_sorted = sorted(low_laplacian_neighbours_candidates, key = lambda x: x[-2])
    y_sorted = sorted(low_laplacian_neighbours_candidates, key = lambda x: x[-1])
    
    low_laplacian_neighbours = x_sorted[:2] + y_sorted[:2]
    return [each[:4] for each in low_laplacian_neighbours]


def calculate_neighbour_score(neighbours):
    neighbours = np.array(neighbours)
    d = neighbours[:, 2]
    w = (1 / d) / sum(1 / d)
    indices = [int(each) for each in neighbours[:, -1]]
    cnt_neighbours = cnt[:, indices] * w
    s = np.sum(cnt_neighbours, axis=1)
    cnt_final = cnt[:, 28] - s
    return cnt_final


def laplacian_filter(ch_names, mode='low'):
    laplaced_channels = []
    for each in ch_names:
        channel_idx = channels.index(each)
        ci = [x[channel_idx], y[channel_idx]]
        dis = calculate_euclidean_distance(channel_poses, ci)
        low_laplacian_neighbours = find_laplacian_neighbours(channel_poses, dis, mode=mode)
        laplaced_channels.append(calculate_neighbour_score(low_laplacian_neighbours))
    return np.array(laplaced_channels).T


def car_filter(the_cnt):
    ref = np.mean(the_cnt, axis=1)
    cnt_localized = the_cnt - np.mean(the_cnt, axis=1).reshape((the_cnt.shape[0], 1))
    return cnt_localized

def select_filts(filt, col_num):

    temp = np.shape(filt)
    columns = np.arange(0,col_num)
    #print(columns)
    f = filt[:, columns]
    for ij in range(col_num):
        f[:, ij] = f[:, ij]/np.linalg.norm(f[:, ij])

    return f


def apply_CCACSP(X, f, col_num):

    f = np.transpose(f)

    temp = np.shape(X)
    num_trials = temp[0]

    #dat = np.zeros(np.shape(X), dtype = object)
    dat = np.zeros((num_trials, 2*col_num, temp[2]))
    for ik in range(num_trials):
        dat[ik,:,:] = np.matmul(f,X[ik,:,:])

    return dat

def my_cov(X, Y):
	avg_X = np.mean(X, axis=1)
	avg_Y = np.mean(Y, axis=1)

	X = X - avg_X[:,None]
	Y = Y - avg_Y[:,None]

	return np.matmul(X, Y.transpose())
 

def calc_CCACSP(x1,x2, numFilt):
    
    num_trials_1 = np.size(x1,0) 
    num_trials_2 = np.size(x2,0) 

    # number of channels and time samples should be the same between x1 and x2
    n_samps = np.size(x1,2)
    n_chans = np.size(x1,1) 

    c1_shifted = np.zeros([n_chans,n_chans])
    c2_shifted = np.zeros([n_chans,n_chans])
    c1 = np.zeros([n_chans,n_chans])
    c2 = np.zeros([n_chans,n_chans])

    range0 = range(0,n_samps-2)
    range1 = range(1,n_samps-1)
    range2 = range(2,n_samps)

    # estimate the covariances 
    for ik in range(num_trials_1):
        Samp = x1[ik]
        temp1 = 0.5*(Samp[:,range0]+Samp[:,range2])
        temp2 = Samp[:,range1]

        c1_shifted = c1_shifted+ my_cov(temp2, temp1)/np.trace(my_cov(temp2, temp1))
        c1 = c1+np.cov(x1[ik])/np.trace(np.cov(x1[ik]))

    c1_shifted = np.divide(c1_shifted,num_trials_1)
    c1 = np.divide(c1,num_trials_1)

    for ik in range(num_trials_2):
        Samp = x2[ik]
        temp1 = 0.5*(Samp[:,range0]+Samp[:,range2])
        temp2 = Samp[:,range1]
	
        c2_shifted = c2_shifted+ my_cov(temp2, temp1)/np.trace(my_cov(temp2, temp1))
        c2 = c2+np.cov(x2[ik])/np.trace(np.cov(x2[ik]))

    c2_shifted = np.divide(c2_shifted,num_trials_2)
    c2 = np.divide(c2,num_trials_2)
        

    # taking care of rank deficiency for a more robust result 
    D, V = eigh(c1+c2) 
    indx = np.argsort(D)
    indx = indx[::-1]
    d = D[indx[0:np.linalg.matrix_rank(c1+c2)]]
    W = V[:,indx[0:np.linalg.matrix_rank(c1+c2)]]
    W_T = np.matmul(np.sqrt(pinv(np.diag(d))),W.transpose())

    S1 = np.matmul(np.matmul(W_T,c1),W_T.transpose())
    S2 = np.matmul(np.matmul(W_T,c2),W_T.transpose())
    S1_shifted = np.matmul(np.matmul(W_T,c1_shifted),W_T.transpose())
    S2_shifted = np.matmul(np.matmul(W_T,c2_shifted),W_T.transpose())

    # find filters for class 1
    d,v = eigh(S1_shifted,S1+S2)
    indx = np.argsort(d)
    indx = indx[::-1]
    filts_1 = v.take(indx, axis=1)
    filts_1 = np.matmul(filts_1.transpose(),W_T)
    filts_1 = filts_1.transpose()
    filts_1 = select_filts(filts_1, numFilt)

    # find filters for class 2
    d,v = eigh(S2_shifted,S1+S2)
    indx = np.argsort(d)
    indx = indx[::-1]
    filts_2 = v.take(indx, axis=1)
    filts_2 = np.matmul(filts_2.transpose(),W_T)
    filts_2 = filts_2.transpose()
    filts_2 = select_filts(filts_2, numFilt)

    
    # concatenate filters for classes 1 and 2 and return 
    return np.concatenate((filts_1, filts_2), axis=1)

In [11]:
bands = np.array([
         [4, 8, 12, 16, 20, 24, 28, 32, 36],
         [8, 12, 16, 20, 24, 28, 32, 36, 40]
])

In [12]:
all_train_features_0, all_train_features_1 = [], []
all_test_features_0, all_test_features_1 = [], []

for i in range(bands.shape[1]):
    # ================================== select bands

    wn = bands[:, i]
    cnt_banded = butter_bandpass_filter(cnt, wn[0], wn[1], fs, 3)
    cnt_localized = car_filter(cnt_banded)

    left_trials, foot_trials = np.zeros((400, 59, 100)), np.zeros((400, 59, 100))
    c1, c2 = 0, 0
    for i in range(len(trials)):
        trial_signal = np.array(cnt_localized[trials[i]:trials[i] + 400, :])
        if labels[i] == 1:
            left_trials[:, :, c1] = trial_signal
            c1 += 1
        elif labels[i] == -1:    
            foot_trials[:, :, c2] = trial_signal
            c2 += 1
    left_trials, foot_trials = np.array(left_trials), np.array(foot_trials)

    # ================================== Define sets
    
    X = list(left_trials.T) + list(foot_trials.T)
    y = [0] * left_trials.T.shape[0] + [1] * foot_trials.T.shape[0]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=1)
    X = np.array(X)
    y = np.array(y)
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)

    # ================================== Extract labels

    X_train_0, X_train_1 = [], []
    for i in range(y_train.shape[0]):
        if y_train[i] == 0:
            X_train_0.append(X_train[i, :, :])
        elif y_train[i] == 1:
            X_train_1.append(X_train[i, :, :])
    X_train_0 = np.array(X_train_0)
    X_train_1 = np.array(X_train_1)

    X_test_0, X_test_1 = [], []
    for i in range(X_test.shape[0]):
        if y_test[i] == 0:
            X_test_0.append(X_test[i, :, :])
        elif y_test[i] == 1:
            X_test_1.append(X_test[i, :, :])
    X_test_0 = np.array(X_test_0)
    X_test_1 = np.array(X_test_1)

    # ================================== CSP

    number_of_filters = 1
    W = calc_CCACSP(X_train_0, X_train_1, number_of_filters)

    # ================================== Apply CSP
    
    X_train_0_features, X_train_1_features = [], []
    for i in range(X_train_0.shape[0]):
        trial = X_train_0[i, :, :]
        filtered_trial = np.dot(W.T, trial)
        f = np.var(filtered_trial.T, axis=0)
        # f = np.log10(np.var(filtered_trial.T, axis=0) / np.sum(np.var(filtered_trial.T, axis=0)))
        X_train_0_features.append(f)
    for i in range(X_train_1.shape[0]):
        trial = X_train_1[i, :, :]
        filtered_trial = np.dot(W.T, trial)
        f = np.var(filtered_trial.T, axis=0)
        # f = np.log10(np.var(filtered_trial.T, axis=0) / np.sum(np.var(filtered_trial.T, axis=0)))
        X_train_1_features.append(f)

    X_train_0_features = np.array(X_train_0_features).T
    X_train_1_features = np.array(X_train_1_features).T

    X_test_0_features, X_test_1_features = [], []
    for i in range(X_test_0.shape[0]):
        trial = X_test_0[i, :, :]
        filtered_trial = np.dot(W.T, trial)
        f = np.var(filtered_trial.T, axis=0)
        # f = np.log10(np.var(filtered_trial.T, axis=0) / np.sum(np.var(filtered_trial.T, axis=0)))
        X_test_0_features.append(f)
    for i in range(X_test_1.shape[0]):
        trial = X_test_1[i, :, :]
        filtered_trial = np.dot(W.T, trial)
        f = np.var(filtered_trial.T, axis=0)
        # f = np.log10(np.var(filtered_trial.T, axis=0) / np.sum(np.var(filtered_trial.T, axis=0)))
        X_test_1_features.append(f)

    X_test_0_features = np.array(X_test_0_features).T
    X_test_1_features = np.array(X_test_1_features).T

    # ================================== Add sets to the super set

    all_train_features_0 += list(X_train_0_features)
    all_train_features_1 += list(X_train_1_features)
    all_test_features_0 += list(X_test_0_features)
    all_test_features_1 += list(X_test_1_features)

all_train_features_0 = np.array(all_train_features_0)
all_train_features_1 = np.array(all_train_features_1)
all_test_features_0 = np.array(all_test_features_0)
all_test_features_1 = np.array(all_test_features_1)

print(all_train_features_0.shape, all_train_features_1.shape, all_test_features_0.shape, all_test_features_1.shape)

(18, 70) (18, 70) (18, 30) (18, 30)


In [13]:
X_train = np.array(list(all_train_features_0.T) + list(all_train_features_1.T))
y_train = np.array([0] * all_train_features_0.shape[1] + [1] * all_train_features_1.shape[1])
X_train, y_train = shuffle(X_train, y_train)

X_test = np.array(list(all_test_features_0.T) + list(all_test_features_1.T))
y_test = np.array([0] * all_test_features_0.shape[1] + [1] * all_test_features_1.shape[1])
X_test, y_test = shuffle(X_test, y_test)

print('X_train:', X_train.shape, 'y_train:', y_train.shape)
print('X_test:', X_test.shape, 'y_test:', y_test.shape)

X_train: (140, 18) y_train: (140,)
X_test: (60, 18) y_test: (60,)


KNN

In [14]:
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.97      0.95        30
           1       0.97      0.93      0.95        30

    accuracy                           0.95        60
   macro avg       0.95      0.95      0.95        60
weighted avg       0.95      0.95      0.95        60



In [15]:
final_features_size = 10
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean') 
sfs = SequentialFeatureSelector(knn, n_features_to_select=final_features_size)
sfs.fit(X_train, y_train)
X_train_knn = sfs.transform(X_train)
X_test_knn = sfs.transform(X_test)
knn.fit(X_train_knn, y_train)
y_pred = knn.predict(X_test_knn)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.90      0.93        30
           1       0.91      0.97      0.94        30

    accuracy                           0.93        60
   macro avg       0.94      0.93      0.93        60
weighted avg       0.94      0.93      0.93        60



SVM

In [16]:
clf = svm.SVC(kernel='poly')
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
X_train_svm = scaling.transform(X_train)
X_test_svm = scaling.transform(X_test)
clf.fit(X_train_svm, y_train)
y_pred = clf.predict(X_test_svm)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.93      0.81        30
           1       0.90      0.63      0.75        30

    accuracy                           0.78        60
   macro avg       0.81      0.78      0.78        60
weighted avg       0.81      0.78      0.78        60



In [17]:
final_features_size = 10
clf = svm.SVC(kernel='poly')
sfs = SequentialFeatureSelector(clf, n_features_to_select=final_features_size)
sfs.fit(X_train_svm, y_train) 
X_train_svm = sfs.transform(X_train_svm)
X_test_svm = sfs.transform(X_test_svm)
clf.fit(X_train_svm, y_train)
y_pred = clf.predict(X_test_svm)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.80      0.75        30
           1       0.77      0.67      0.71        30

    accuracy                           0.73        60
   macro avg       0.74      0.73      0.73        60
weighted avg       0.74      0.73      0.73        60



LDA

In [18]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
y_pred = lda.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.90      0.84        30
           1       0.88      0.77      0.82        30

    accuracy                           0.83        60
   macro avg       0.84      0.83      0.83        60
weighted avg       0.84      0.83      0.83        60



In [19]:
final_features_size = 10
lda = LinearDiscriminantAnalysis()
sfs = SequentialFeatureSelector(lda, n_features_to_select=final_features_size)
sfs.fit(X_train, y_train)
X_train_lda = sfs.transform(X_train)
X_test_lda = sfs.transform(X_test)
lda.fit(X_train_lda, y_train)
y_pred = lda.predict(X_test_lda)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.87      0.87        30
           1       0.87      0.87      0.87        30

    accuracy                           0.87        60
   macro avg       0.87      0.87      0.87        60
weighted avg       0.87      0.87      0.87        60



RL

In [62]:
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
X_train_rl = scaling.transform(X_train)
X_test_rl = scaling.transform(X_test)
X_train_rl.shape, X_test_rl.shape

((140, 18), (60, 18))

In [63]:
class SignalEnv(Env):
    def __init__(self, X_set, y_set):
        self.X, self.y = X_set, y_set
        self.action_space = Discrete(2)
        self.observation_space = Box(low=-1, high=1, shape=(self.X[0].shape[0],))
        self.state = 0
        
    def step(self, action):
        
        # Calculate reward
        if self.y[self.state] == action: 
            reward = 1 
        else: 
            reward = -1 
        
        info = {}
        self.state = self.state + 1 

        # Check if done
        if self.state >= len(self.X): 
            return _, reward, True, info
        else:
            return np.array(self.X[self.state]), reward, False, info

    def render(self):
        pass
    
    def reset(self):
        self.state = 0
        return np.array(self.X[self.state])

In [64]:
env = SignalEnv(X_train_rl, y_train)
print(env.action_space.sample())
print(env.observation_space.sample())

1
[ 0.02704591 -0.7407108   0.5536843  -0.23932812 -0.7339993   0.9175148
  0.8590896  -0.8533594  -0.76906174  0.01184877 -0.6061407   0.331069
  0.46584174 -0.80483234 -0.8158638  -0.60256946  0.53478116  0.5820849 ]


In [65]:
episodes = 5
for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
    
env.close()

Episode:1 Score:8
Episode:2 Score:-6
Episode:3 Score:4
Episode:4 Score:4
Episode:5 Score:6


In [70]:
log_path = '/content/drive/MyDrive/Brain-Computer Interfaces/logs/'
model = DQN("MlpPolicy", env, tensorboard_log=log_path)
model.learn(total_timesteps=700000)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


(36.0, 0.0)

In [ ]:
model.save('/content/drive/MyDrive/Brain-Computer Interfaces/Saved Models (RL)/PPO_2')

In [82]:
attempts, corrects = 0, 0
y_pred = []
env = SignalEnv(X_test_rl, y_test)
obs = env.reset()
cc = 0
while True:
    action, _ = model.predict(obs) 
    obs, reward, done, info = env.step(action)
    y_pred.append(action)
    attempts += 1
    if reward > 0:
        corrects += 1
    if done: 
        print('info', info)
        break
    cc += 1

print(corrects, attempts)
print('Accuracy:', corrects / len(X_test))

info {}
59 60
Accuracy: 0.9833333333333333


In [84]:
from sklearn.metrics import mean_squared_error

print(classification_report(y_test, y_pred))
print(mean_squared_error(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.97      0.98        30
           1       0.97      1.00      0.98        30

    accuracy                           0.98        60
   macro avg       0.98      0.98      0.98        60
weighted avg       0.98      0.98      0.98        60

0.016666666666666666


### All together

In [25]:
DATASET_PATH = '/content/drive/MyDrive/Brain-Computer Interfaces/Motor Imagery Dataset/'
participants = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
bands = np.array([
         [4, 8, 12, 16, 20, 24, 28, 32, 36],
         [8, 12, 16, 20, 24, 28, 32, 36, 40]
])
model_results = {}
for participant in participants:
    
    class SignalEnv(Env):
        def __init__(self, X_set, y_set):
            self.X, self.y = X_set, y_set
            self.action_space = Discrete(2)
            self.observation_space = Box(low=-1, high=1, shape=(self.X[0].shape[0],))
            self.state = 0
            
        def step(self, action):
            
            # Calculate reward
            if self.y[self.state] == action: 
                reward = 1 
            else: 
                reward = -1 
            
            info = {}
            self.state = self.state + 1 

            # Check if done
            if self.state >= len(self.X): 
                return _, reward, True, info
            else:
                return np.array(self.X[self.state]), reward, False, info

        def render(self):
            pass
        
        def reset(self):
            self.state = 0
            return np.array(self.X[self.state])

    print()
    print(model_results)
    print()
    mat = loadmat(DATASET_PATH + 'BCICIV_calib_ds1' + participant + '_100Hz.mat')
    cnt = mat['cnt'] * 0.1
    mrk = mat['mrk']
    nfo = mat['nfo']
    x = [each[0] for each in nfo['xpos'][0][0]]
    y = [each[0] for each in nfo['ypos'][0][0]]
    trials = mrk[0][0][0][0]
    labels = mrk[0][0][1][0]
    fs = nfo['fs'][0][0][0][0]
    channels = [each[0] for each in nfo['clab'][0][0][0]]
    channel_poses = [[x[i], y[i]] for i in range(len(x))]
    # label_names = [each[0] for each in nfo['classes'][0][0][0]]
    print('cnt.shape', cnt.shape)
    print('trials', trials[:10], 'trials.shape', trials.shape)
    print('labels', labels[:10], 'labels.shape', labels.shape)
    print('fs', fs)
    print('channels', channels)
    all_train_features_0, all_train_features_1 = [], []
    all_test_features_0, all_test_features_1 = [], []

    for i in range(bands.shape[1]):
        # ================================== select bands

        wn = bands[:, i]
        cnt_banded = butter_bandpass_filter(cnt, wn[0], wn[1], fs, 3)
        cnt_localized = car_filter(cnt_banded)

        left_trials, foot_trials = np.zeros((400, 59, 100)), np.zeros((400, 59, 100))
        c1, c2 = 0, 0
        for i in range(len(trials)):
            trial_signal = np.array(cnt_localized[trials[i]:trials[i] + 400, :])
            if labels[i] == 1:
                left_trials[:, :, c1] = trial_signal
                c1 += 1
            elif labels[i] == -1:    
                foot_trials[:, :, c2] = trial_signal
                c2 += 1
        left_trials, foot_trials = np.array(left_trials), np.array(foot_trials)

        # ================================== Define sets
        
        X = list(left_trials.T) + list(foot_trials.T)
        y = [0] * left_trials.T.shape[0] + [1] * foot_trials.T.shape[0]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=1)
        X = np.array(X)
        y = np.array(y)
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        X_test = np.array(X_test)
        y_test = np.array(y_test)

        # ================================== Extract labels

        X_train_0, X_train_1 = [], []
        for i in range(y_train.shape[0]):
            if y_train[i] == 0:
                X_train_0.append(X_train[i, :, :])
            elif y_train[i] == 1:
                X_train_1.append(X_train[i, :, :])
        X_train_0 = np.array(X_train_0)
        X_train_1 = np.array(X_train_1)

        X_test_0, X_test_1 = [], []
        for i in range(X_test.shape[0]):
            if y_test[i] == 0:
                X_test_0.append(X_test[i, :, :])
            elif y_test[i] == 1:
                X_test_1.append(X_test[i, :, :])
        X_test_0 = np.array(X_test_0)
        X_test_1 = np.array(X_test_1)

        # ================================== CSP

        number_of_filters = 1
        W = calc_CCACSP(X_train_0, X_train_1, number_of_filters)

        # ================================== Apply CSP
        
        X_train_0_features, X_train_1_features = [], []
        for i in range(X_train_0.shape[0]):
            trial = X_train_0[i, :, :]
            filtered_trial = np.dot(W.T, trial)
            f = np.var(filtered_trial.T, axis=0)
            # f = np.log10(np.var(filtered_trial.T, axis=0) / np.sum(np.var(filtered_trial.T, axis=0)))
            X_train_0_features.append(f)
        for i in range(X_train_1.shape[0]):
            trial = X_train_1[i, :, :]
            filtered_trial = np.dot(W.T, trial)
            f = np.var(filtered_trial.T, axis=0)
            # f = np.log10(np.var(filtered_trial.T, axis=0) / np.sum(np.var(filtered_trial.T, axis=0)))
            X_train_1_features.append(f)

        X_train_0_features = np.array(X_train_0_features).T
        X_train_1_features = np.array(X_train_1_features).T

        X_test_0_features, X_test_1_features = [], []
        for i in range(X_test_0.shape[0]):
            trial = X_test_0[i, :, :]
            filtered_trial = np.dot(W.T, trial)
            f = np.var(filtered_trial.T, axis=0)
            # f = np.log10(np.var(filtered_trial.T, axis=0) / np.sum(np.var(filtered_trial.T, axis=0)))
            X_test_0_features.append(f)
        for i in range(X_test_1.shape[0]):
            trial = X_test_1[i, :, :]
            filtered_trial = np.dot(W.T, trial)
            f = np.var(filtered_trial.T, axis=0)
            # f = np.log10(np.var(filtered_trial.T, axis=0) / np.sum(np.var(filtered_trial.T, axis=0)))
            X_test_1_features.append(f)

        X_test_0_features = np.array(X_test_0_features).T
        X_test_1_features = np.array(X_test_1_features).T

        # ================================== Add sets to the super set

        all_train_features_0 += list(X_train_0_features)
        all_train_features_1 += list(X_train_1_features)
        all_test_features_0 += list(X_test_0_features)
        all_test_features_1 += list(X_test_1_features)

    all_train_features_0 = np.array(all_train_features_0)
    all_train_features_1 = np.array(all_train_features_1)
    all_test_features_0 = np.array(all_test_features_0)
    all_test_features_1 = np.array(all_test_features_1)

    print(all_train_features_0.shape, all_train_features_1.shape, all_test_features_0.shape, all_test_features_1.shape)

    X_train = np.array(list(all_train_features_0.T) + list(all_train_features_1.T))
    y_train = np.array([0] * all_train_features_0.shape[1] + [1] * all_train_features_1.shape[1])
    X_train, y_train = shuffle(X_train, y_train)

    X_test = np.array(list(all_test_features_0.T) + list(all_test_features_1.T))
    y_test = np.array([0] * all_test_features_0.shape[1] + [1] * all_test_features_1.shape[1])
    X_test, y_test = shuffle(X_test, y_test)

    print('X_train:', X_train.shape, 'y_train:', y_train.shape)
    print('X_test:', X_test.shape, 'y_test:', y_test.shape)

    # KNN
    print('participant:', participant, '(KNN)')
    knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    print(classification_report(y_test, y_pred))
    model_results['knn1'] = classification_report(y_test, y_pred)
    final_features_size = 10
    knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean') 
    sfs = SequentialFeatureSelector(knn, n_features_to_select=final_features_size)
    sfs.fit(X_train, y_train)
    X_train_knn = sfs.transform(X_train)
    X_test_knn = sfs.transform(X_test)
    knn.fit(X_train_knn, y_train)
    y_pred = knn.predict(X_test_knn)
    print(classification_report(y_test, y_pred))
    model_results['knn2'] = classification_report(y_test, y_pred)

    # SVM
    print('participant:', participant, '(SVM)')
    clf = svm.SVC(kernel='poly')
    scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
    X_train_svm = scaling.transform(X_train)
    X_test_svm = scaling.transform(X_test)
    clf.fit(X_train_svm, y_train)
    y_pred = clf.predict(X_test_svm)
    print(classification_report(y_test, y_pred))
    model_results['svm1'] = classification_report(y_test, y_pred)
    final_features_size = 10
    clf = svm.SVC(kernel='poly')
    sfs = SequentialFeatureSelector(clf, n_features_to_select=final_features_size)
    sfs.fit(X_train_svm, y_train) 
    X_train_svm = sfs.transform(X_train_svm)
    X_test_svm = sfs.transform(X_test_svm)
    clf.fit(X_train_svm, y_train)
    y_pred = clf.predict(X_test_svm)
    print(classification_report(y_test, y_pred))
    model_results['svm2'] = classification_report(y_test, y_pred)

    # LDA
    print('participant:', participant, '(LDA)')
    lda = LinearDiscriminantAnalysis()
    lda.fit(X_train, y_train)
    y_pred = lda.predict(X_test)
    print(classification_report(y_test, y_pred))
    model_results['lda1'] = classification_report(y_test, y_pred)
    final_features_size = 10
    lda = LinearDiscriminantAnalysis()
    sfs = SequentialFeatureSelector(lda, n_features_to_select=final_features_size)
    sfs.fit(X_train, y_train)
    X_train_lda = sfs.transform(X_train)
    X_test_lda = sfs.transform(X_test)
    lda.fit(X_train_lda, y_train)
    y_pred = lda.predict(X_test_lda)
    print(classification_report(y_test, y_pred))
    model_results['lda2'] = classification_report(y_test, y_pred)

    # RL
    print('participant:', participant, '(RL)')
    scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
    X_train_rl, X_test_rl = None, None
    X_train_rl = scaling.transform(X_train)
    X_test_rl = scaling.transform(X_test)
    env = None
    env = SignalEnv(X_train_rl, y_train)
    model = None
    log_path = '/content/drive/MyDrive/Brain-Computer Interfaces/logs/'
    model = PPO("MlpPolicy", env, tensorboard_log=log_path)
    model.learn(total_timesteps=700000)
    attempts, corrects = 0, 0
    y_pred = []
    env = SignalEnv(X_test_rl, y_test)
    obs = env.reset()
    cc = 0
    while True:
        action, _ = model.predict(obs) 
        obs, reward, done, info = env.step(action)
        y_pred.append(action)
        attempts += 1
        if reward > 0:
            corrects += 1
        if done: 
            print('info', info)
            break
        cc += 1

    print(corrects, attempts)
    print('Accuracy:', corrects / len(X_test))
    model_results['RL1'] = classification_report(y_test, y_pred)
    model_results['RL2'] = mean_squared_error(y_test,y_pred)


{}

cnt.shape (190594, 59)
trials [2091 2891 3691 4491 5291 6091 6891 7691 8491 9291] trials.shape (200,)
labels [ 1  1 -1  1  1  1  1 -1 -1  1] labels.shape (200,)
fs 100
channels ['AF3', 'AF4', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'CFC7', 'CFC5', 'CFC3', 'CFC1', 'CFC2', 'CFC4', 'CFC6', 'CFC8', 'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8', 'CCP7', 'CCP5', 'CCP3', 'CCP1', 'CCP2', 'CCP4', 'CCP6', 'CCP8', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'PO1', 'PO2', 'O1', 'O2']
(18, 70) (18, 70) (18, 30) (18, 30)
X_train: (140, 18) y_train: (140,)
X_test: (60, 18) y_test: (60,)
participant: a (KNN)
              precision    recall  f1-score   support

           0       0.62      0.67      0.65        30
           1       0.64      0.60      0.62        30

    accuracy                           0.63        60
   macro avg       0.63      0.63      0.63        60
weighted avg      